Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:

import logging
from itertools import combinations

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from icecream import ic

Cost: $d + (d \cdot \alpha \cdot w)^\beta$ with $\alpha \ge 0$ and $\beta \ge 0$

## ACO 

In [3]:
import numpy as np
import networkx as nx

class ACO_Solver:
    def __init__(self, problem, n_ants=15, n_iterations=50, alpha_aco=1.0, beta_aco=3.0, rho=0.2):
        self.prob = problem
        self.graph = problem.graph
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha_aco = alpha_aco
        self.beta_aco = beta_aco
        self.rho = rho
        
        # Precalcoliamo i cammini minimi per "sapere dove andare"
        self.all_pairs_dist = dict(nx.all_pairs_dijkstra_path_length(self.graph, weight='dist'))
        self.dist_to_depot = nx.shortest_path_length(self.graph, target=0, weight='dist')
        
        # tutti i feronomoni inizializzati a 1.0, stessa probabilità
        self.pheromones = {edge: 1.0 for edge in self.graph.edges()}
        for (u, v) in list(self.pheromones.keys()):
            self.pheromones[(v, u)] = 1.0

    def get_eta(self, u, v, current_weight, visited_gold, current_trip_cost):
        # quanto è "buono" andare da u a v
           
        d_uv = self.graph[u][v]['dist']
        
        if current_weight == 0:
            cost_imm = current_trip_cost + d_uv
            #print("current_weight is zero, adjusting eta calculation, portiamo l'esplorazione verso città con oro.")
            # cerca la città raggiungibile con oro più vicina
            gold_cities = [n for n in self.graph.nodes if n not in visited_gold and self.graph.nodes[n]['gold'] > 0]
            if gold_cities:
                nearest_gold = min(gold_cities, key=lambda x: self.all_pairs_dist[u][x])
                d_to_gold = self.all_pairs_dist[u][nearest_gold]
                cost_andata = cost_imm + d_to_gold  # sommo al trip totale la distanza per arrivare all'oro (senza carico)
                # dalla città con oro a tornare al deposito, considerando il peso dell'oro raccolto
                distanza_ritorno= self.dist_to_depot[nearest_gold]
                cost_ritorno= distanza_ritorno + (self.prob.alpha * distanza_ritorno * self.graph.nodes[nearest_gold]['gold']) ** self.prob.beta
                return 1.0 / (cost_andata + cost_ritorno + 1e-9)
            else:
                # nessuna città con oro rimasta, procedi normalmente
                print("No more gold cities reachable from node {}.".format(u))
                return 0
        else:
            # stiamo già trasportando dell'oro
            cost_imm = d_uv + (self.prob.alpha * d_uv * current_weight) ** self.prob.beta
            # di ritorno al deposito
            # raccolgo l'oro nella città v se non l'ho già fatto e considero il costo di tornare indietro 
            gold_v= self.graph.nodes[v]['gold'] if v not in visited_gold else 0
            new_weight = current_weight + gold_v
            # calcolo costo per tornare al deposito 
            d_deposito = self.all_pairs_dist[v][0]
            cost_fut = d_deposito + (self.prob.alpha * d_deposito * new_weight) ** self.prob.beta

            return 1.0 / (current_trip_cost + cost_imm + cost_fut + 1e-9)


    def _build_ant_path(self):
        current_node = 0
        visited_gold = {0}
        path = [0]
        total_cost = 0
        current_weight = 0
        num_total_nodes = len(self.graph.nodes)
        print("FORMICA NUOVA")
        print("0-> ", end="")
        
        current_trip_cost=0
        current_trip=[0]

        while len(visited_gold) < num_total_nodes:
            neighbors = list(self.graph.neighbors(current_node))
            
            # --- SOLUZIONE AL LOOP ---
            # Selezioniamo solo i vicini che portano verso l'oro o che sono "nuovi"
            # Impediamo il ritorno immediato al nodo precedente se non c'è oro
            valid_neighbors = []
            for n in neighbors:
                # Non tornare al deposito se non abbiamo oro (già presente)
                if n == 0 and current_weight == 0: continue

                # se il carico è zero, non torniamo indietro
                if current_weight == 0 and n in current_trip: continue


                # Evita di rimbalzare tra due nodi senza oro
                if len(path) > 1 and n == path[-2] and current_node in visited_gold:
                    continue
                valid_neighbors.append(n)
            
            if not valid_neighbors: 
                print("Warning: no valid neighbors, relaxing constraints.")
                valid_neighbors = neighbors

            probs = []
            for n in valid_neighbors:
                edge = (current_node, n) if (current_node, n) in self.pheromones else (n, current_node)
                tau = self.pheromones[edge]
                eta = self.get_eta(current_node, n, current_weight, visited_gold, current_trip_cost)
                
                # A COSA SERVONO ALPHA ACO E BETA ACO?
                probs.append((tau ** self.alpha_aco) * (eta ** self.beta_aco) )

            probs = np.array(probs) / sum(probs)
            next_node = np.random.choice(valid_neighbors, p=probs)

            # Calcolo costo e aggiornamento
            d = self.graph[current_node][next_node]['dist']
            current_trip_cost += d + (self.prob.alpha * d * current_weight) ** self.prob.beta
            total_cost += d + (self.prob.alpha * d * current_weight) ** self.prob.beta

            
            if next_node not in visited_gold:
                # Raccogli l'oro
                # NOTA posso valutare di prenderne solo una parte quando vantaggioso, PER DOPO
                current_weight += self.graph.nodes[next_node]['gold']
                visited_gold.add(next_node)

                
            
            current_node = next_node
            print(str(current_node), end="")

            
            if current_node ==0 :
                print("\n Ritornato al deposito, peso attuale: {}, costo totale finora: {}".format(current_weight, total_cost))
                current_weight = 0  # svuota il peso al deposito 
                current_trip_cost=0
                current_trip=[0]
                print("0-> ", end="")
            else:
                print("->", end="")
                current_trip.append(current_node)
                
            path.append(current_node)
            

        # Ritorno forzato al deposito
        if current_node != 0:
            ret_path = nx.shortest_path(self.graph, source=current_node, target=0, weight='dist')
            for i in range(len(ret_path)-1):
                u, v = ret_path[i], ret_path[i+1]
                total_cost += self.prob.cost([u, v], current_weight) # Usiamo la funzione del problema
                path.append(v)

        return path, total_cost

    def _update_pheromones(self, paths, costs):
        # Evaporazione
        # gli edge migliori (sopra la media) ricevono un contributo proporzionale alla qualità del percorso (1/costo), più il feromone è alto più è probabile che venga scelto, ma anche l'eta (inverso del costo) influenza la scelta
        for edge in self.pheromones: self.pheromones[edge] *= (1 - self.rho)
        # Consideriamo solo le formiche migliori (sopra la media) per il deposito
        avg_cost = np.mean(costs)
        for path, cost in zip(paths, costs):
            if cost <= avg_cost:
                contribution = 1.0 / (cost + 1e-6)
                for i in range(len(path)-1):
                    u, v = path[i], path[i+1]
                    if (u, v) in self.pheromones: self.pheromones[(u, v)] += contribution
                    elif (v, u) in self.pheromones: self.pheromones[(v, u)] += contribution

    def solve(self):
        best_path, best_cost = None, float('inf')
        for gen in range(self.n_iterations):
            results = [self._build_ant_path() for _ in range(self.n_ants)]
            paths, costs = zip(*results)
            min_idx = np.argmin(costs)
            if costs[min_idx] < best_cost:
                best_cost, best_path = costs[min_idx], paths[min_idx]
            self._update_pheromones(paths, costs)
            if gen % 10 == 0: print(f"Gen {gen}: Costo {best_cost:.2f}")
        return best_path, best_cost


## Genetico

In [ ]:
from importlib.resources import path
import numpy as np
import networkx as nx
import random

class GA_Solver:

    def __init__(self, problem, pop_size=50, generations=100, offprint=20):
        self.prob = problem
        self.graph = problem.graph
        # Tutte le città tranne il deposito
        self.cities_to_visit = [n for n in self.graph.nodes if n != 0]
        # Precalcolo cammini minimi
        self.all_paths = dict(nx.all_pairs_dijkstra_path(self.graph, weight='dist'))
        self.pop_size = pop_size
        self.generations = generations
        self.offprint = offprint

    def _calc_path_cost(self, path : list[int], weight: float) -> float:
        c = 0
        current_w = weight
        for u, v in zip(path, path[1:]):
            d = self.graph[u][v]['dist']
            # Formula del costo: d + (alpha * d * w)^beta
            c += d + (self.prob.alpha * d * current_w) ** self.prob.beta
        return c

    def _path_cost(self, path: list[tuple[int, float]]) -> float:
        total_cost = 0
        current_weight = 0
        for i in range(len(path)-1):
            u, v = path[i][0], path[i+1][0]
            d = self.graph[u][v]['dist']
            total_cost += d + (self.prob.alpha * d * current_weight) ** self.prob.beta
            if v==0:
                current_weight = 0  # Svuota il peso al deposito
            else:
                current_weight += path[i+1][1]  # aggiungo oro raccolto al peso
        return total_cost  
    
    def greedy_initialization(self)-> tuple[list[tuple[int, float]], float]:
        """Costruisce un individuo iniziale usando una strategia greedy."""

        if self.prob.beta >= 1.0 :
            # Se beta è alto, il costo del peso è molto penalizzante, quindi conviene fare più viaggi con carichi leggeri
            # potrebbe convenire addirittura portare poco oro per volta (prendere solo una parte dell'oro disponibile in ogni città) e fare più giri
            #print(f"Greedy initialization con beta alto: favorisco più viaggi con carichi leggeri. {self.prob.beta}")
            ind = list(self.cities_to_visit)
            random.shuffle(ind)
            chromo, cost= self._evaluate_and_segment(ind)
            chromo, cost= self.multiple_cycle(chromo)

        else:
            # Se beta è basso, il costo del peso è meno penalizzante, quindi conviene fare meno viaggi con carichi più pesanti:
            ind = list(self.cities_to_visit)
            random.shuffle(ind)
            chromo, cost= self._evaluate_and_segment(ind)

        return chromo, cost
    
    def multiple_cycle (self, chromo: list[tuple[int, float]])-> tuple[list[tuple[int, float]], float]:
        tours = []
        current_tour_cities = []
        #print(chromo)  
        # 1. Dividiamo il percorso originale in tour che partono e tornano a zero (escludendo gli 0)
        for n, w in chromo:
            if n == 0:
                if current_tour_cities:
                    #print("Tour trovato:", current_tour_cities)
                    tours.append(current_tour_cities)
                    current_tour_cities = []
            else:
                current_tour_cities.append((n,w))
        
        trips=[]

        for t in tours:
          closed_tour= [(0,0)]+ t+ [(0,0)]
          cost= self._path_cost( closed_tour)
          #print(closed_tour)
          # simulo il costo di fare più giri in uno stesso tour
          #print(f"costo tour {cost}")
          set_t= [ (c,w) for c,w in t if c!=0 and w!=0]
          
          best_trip=closed_tour

          for i in range(2, int (min(set_t, key=lambda x: x[1])[1] ) ):
              #print(f"Divido il carico per {i}")
              trip=[(0,0)]
              for j in range(i):
                # raccogliere tutto l'oro capire come fare se non è divisibile 
                r= [ 0 for _ in t]
                if j== i-1:
                    r= [ w % i for c, w in t] # se non è divisibile, aggiungo il resto all'ultimo giro
                trip.extend( [ (c, w//i + r) for (c, w), r in zip(t, r) ]) 
                trip.extend([(0,0)])
              cost_trip= self._path_cost( trip)       
              if cost_trip < cost and cost_trip>0:
                  cost= cost_trip
                  best_trip= trip
              else:
                  break
          #print (f"Best trip per questo tour: {best_trip} con costo {cost}")
          trips.extend(best_trip[:-1])  # escludiamo l'ultimo 0 che sarà aggiunto dal tour successivo
        
        trips.append((0,0))  # aggiungiamo l'ultimo ritorno al deposito alla fine di tutti i tour
        #print(f"Trips dopo ottimizzazione multi-cycle: {trips}")
        total_cost= self._path_cost( trips) 
        #print(f"Costo totale dopo ottimizzazione multi-cycle: {total_cost}")
        return trips, total_cost



    def _evaluate_and_segment(self, chromosome: list[int]) -> tuple[list[tuple[int, float]], float]:
        """Decoder Greedy: decide se tornare al deposito per scaricare."""
        route = []
        total_cost = 0
        current_node = 0
        current_weight = 0

        route.append((0, 0)) 
        for next_target in chromosome:
            # Opzione A: Diretto
            path_direct = self.all_paths[current_node][next_target]
            cost_direct = self._calc_path_cost(path_direct, current_weight)
            
            # Opzione B: Passando dal deposito
            path_to_depot = self.all_paths[current_node][0]
            path_from_depot = self.all_paths[0][next_target]
            cost_unload = (self._calc_path_cost(path_to_depot, current_weight) + 
                           self._calc_path_cost(path_from_depot, 0))
            
            if current_weight > 0 and cost_unload < cost_direct:
                # Scarico al deposito
                for v in path_to_depot[1:]:
                    route.append((v, 0))
                current_weight = 0
                # Vado al target
                for v in path_from_depot[1:]:
                    gold = self.graph.nodes[v].get('gold', 0) if v == next_target else 0
                    route.append((v, gold))
                    current_weight += gold
                total_cost += cost_unload
            else:
                # Vado diretto
                for v in path_direct[1:]:
                    gold = self.graph.nodes[v].get('gold', 0) if v == next_target else 0
                    route.append((v, gold))
                    current_weight += gold
                total_cost += cost_direct
            current_node = next_target

        # Ritorno finale
        path_home = self.all_paths[current_node][0]
        total_cost += self._calc_path_cost(path_home, current_weight)
        for node in path_home[1:]:
            route.append((node, 0))
            
        return route, total_cost

    def _create_individual(self) -> tuple[list[tuple[int, float]], float]:
        ind = list(self.cities_to_visit)
        random.shuffle(ind)
        return self._evaluate_and_segment(ind)



    def _mutate(self, ind: list[tuple[int, float]] ) -> tuple[list[tuple[int, float]], float]:
        """
        Mutation: 2 possibilità 
        1. Scambio casuale di due città nel percorso (swap mutation).
        2. Ottimizzazione locale: Scompone il percorso in tour (segmenti tra i depositi) e li ri-decodifica.
        """
        #print(ind)
        
        ratio= random.random()

        if ratio < 0.3:
            # OPZIONE 1:
            # Swap mutation
            chromo = set([n for n, g in ind if n != 0])
            chromo= list(chromo)  # rimuoviamo eventuali duplicati
            # Applichiamo uno swap casuale per diversità
            if len(chromo) >= 2:
                i, j = random.sample(range(len(chromo)), 2)
                chromo[i], chromo[j] = chromo[j], chromo[i]
            
            chromo, cost= self._evaluate_and_segment(chromo)
            if self.prob.beta >= 1.0:
                chromo, cost= self.multiple_cycle(chromo)
            return chromo, cost
        
        else:
            # OPZIONE 2:
            # Ottimizzazione locale: ri-decodifica dei tour
            tours = []
            current_tour_cities = []
                
            # 1. Dividiamo il percorso originale in tour che partono e tornano a zero (escludendo gli 0)
            for n, w in ind:
                if n == 0:
                    if current_tour_cities:
                        #print("Tour trovato:", current_tour_cities)
                        tours.append(current_tour_cities)
                        current_tour_cities = []
                else:
                    current_tour_cities.append(n)
                        
            # Aggiungiamo l'ultimo tour se il percorso non finiva con 0
            if current_tour_cities:
                #print("Tour trovato:", current_tour_cities)
                tours.append(current_tour_cities)

            #print("Eseguo ottimizzazione locale sui tour")
            mutated_ind = []
            mutated_ind.append((0, 0))
            # Il percorso deve iniziare con il deposito
            golden_city= set()
            # 2. Per ogni tour trovato, proviamo a ri-eseguire il decoder greedy
            for t in tours:
                # t è una lista di soli nomi di città (interi)
                # escludiamo le città dove ho già raccolto l'oro
                t = [city for city in t if city not in golden_city]
                # trasformiamo t in un set
                t_set = set(t)
                #print ("Tour da ri-decodificare (escludendo città già raccolte):", t_set)
                if len(t_set) > 0:
                    # Re-decodifichiamo il segmento di città
                    new_segment_route, _ = self._evaluate_and_segment(t_set)
                    #print("Nuovo segmento dopo ri-decodifica:", new_segment_route)
                    mutated_ind.extend(new_segment_route[1:])  # escludiamo il primo 0 del segmento, già presente
                    
                    # Aggiorniamo l'insieme delle città dove abbiamo raccolto l'oro
                    golden_city.update(t_set)

            if self.prob.beta >= 1.0:
                mutated_ind, cost= self.multiple_cycle(mutated_ind)
            else:
                cost= self._path_cost(mutated_ind)
                
            return mutated_ind, cost
    
    def _optimize_tour(self, tour: list[tuple[int, float]]) -> list[tuple[int, float]]:
        # Ottimizzazione locale per un singolo tour (escluso il deposito)
        target_city = [(c, w) for c, w in tour if c != 0 and w > 0]
        
        # Creare un dizionario per mappare città -> oro originale
        city_to_gold = {c: self.graph.nodes[c]['gold'] for c, w in target_city}
        
        # Controllo se esiste un percorso più breve per raccogliere l'oro in queste città
        optimize_tour, _ = self._evaluate_and_segment([c for c, w in target_city])
        
        new_tour = []
        for c, w in optimize_tour:
            # Usa il dizionario per ottenere l'oro originale della città
            real_w = city_to_gold.get(c, 0)
            if real_w:
                new_tour.append((c, real_w))
            else:
                new_tour.append((c, 0))
        
        return new_tour

    
    def _crossover(self, parent1: list[tuple[int, float]], parent2: list[tuple[int, float]]) -> tuple[list[tuple[int, float]], float]:
        # Crossover: 
        # estraggo dal primo genitore i tour che completano il raccoglimento di oro per le prime k= tot city/ 2 città e le restanti estraggo i tour dal genitore 2
        
        k= len(self.cities_to_visit) // 2
        tours_p1= []
        collect_gold_in_p1 = {} 
        tour=[]
        # ciclo che estrae dal primo genitore i tour che completano il raccoglimento di oro per le prime k città e le restanti estraggo dal genitore 2
        
        for c, w in parent1[1:]: 
            if c==0:
                # città deposito tour concluso
                if tour:
                    tours_p1.extend([(0,0)]+tour)
                    tour=[]
                # esco dal loop quando quando ho raccolto l'oro in tutte le k città
                if len(collect_gold_in_p1) >= k and all(v == 0 for v in collect_gold_in_p1.values()):
                    break
            else:
                # se la cittè c non è ancora stata visitata e c'è ancora spazio (meno di k città) la inserisco in collect_gold_in_p1
                if c not in collect_gold_in_p1 and w>0 and len(collect_gold_in_p1)<k:
                    # inserisco tuple (c, oro residuo) in collect_gold_in_p1
                    collect_gold_in_p1[c] = self.graph.nodes[c]['gold']-w 
                    tour.append((c,w)) 
                else:
                    # sottrarre il valore w al valore già presente in collect_gold_in_p1 per la città c
                    if c in collect_gold_in_p1:
                        collect_gold_in_p1[c] = max(0, collect_gold_in_p1[c] - w)
                        tour.append((c,w))
                    else:
                        tour.append((c,0))
        #print ("Città raccolte da p1:", collect_gold_in_p1)
        #print("Tour estratti dal genitore 1:", tours_p1)
        # estraggo i tour dal genitore 2 che completano il raccoglimento di oro per le città non ancora raccolte da p1
        tours_p2= []
        # le città che deve raccogliere p2 sono quelle rimanenti 
        collect_gold_in_p2 = {c: self.graph.nodes[c]['gold'] 
                          for c in self.graph.nodes 
                          if c not in collect_gold_in_p1 and c != 0}
        tour=[]
        
        for c, w in parent2[1:]:
            if c==0:
                # città deposito tour concluso
                if tour:
                    # ottimizzare il tour t2 prima di aggiungerlo a tours p2
                    tour = self._optimize_tour(tour) 
                    tours_p2.extend(tour[:-1]) # escludiamo l'ultimo 0 che sarà aggiunto dal tour successivo
                    tour=[]
                # esco dal loop quando quando ho raccolto tutto l'oro
                if all(v == 0 for v in collect_gold_in_p2.values()):
                    break
            else:
                # se la cittè c non è ancora stata visitata e c'è ancora spazio (meno di k città) la inserisco in collect_gold_in_p2
                if c in collect_gold_in_p2 and w>0 :
                    #print(f"Raccolgo oro in città {c} da genitore 2, oro residuo: {collect_gold_in_p2[c] - w}")
                    collect_gold_in_p2[c] = max(0, collect_gold_in_p2[c] - w)
                    tour.append((c,w))
                else:
                    tour.append((c,0))
        
        offspring = tours_p1 + tours_p2 + [(0,0)]  # aggiungo il ritorno finale al deposito
        cost = self._path_cost(offspring)
        #print("Città raccolte da p2:", collect_gold_in_p2)
        #print("Tour estratti dal genitore 2:", tours_p2)
        #print("Offspring dopo crossover:", offspring)
        # print(f"Costo dell'offspring: {cost}")
        return offspring, cost
       

    def _run_ga_logic(self)-> list[tuple[int, float]]:
        population = [self.greedy_initialization() for _ in range(self.pop_size)]
        #print("Popolazione iniziale creata.")
        #print(  [cost for cromo, cost in population] )
        # ordina la popolazione in base al costo (decrescnente)
        population = sorted(population, key=lambda x: x[1])
        best_chromo = population[0][0]
        #print(best_chromo)

        best_cost = population[0][1]
        #print(f"Best cost iniziale: {best_cost}")
        stagnation_counter = 0

        for gen in range(self.generations):
            #print(f"Generazione {gen}, miglior costo: {best_cost}")
            next_gen = []
            for _ in range(self.offprint//2):
                parents = []
                for _ in range(2):
                    candidates = random.sample(population, 2)
                    parents.append(min(candidates, key=lambda x: x[1])[0])
                # per ogni iterazione 2 figli

                # estraggo un valore tra 0 e 1
                ratio = random.random()
                if ratio < 0.7:
                    # mutation
                    # print("MUTAZIONE")
                    #print("Genitore 1:", parents[0])
                    #print("Genitore 2:", parents[1])
                    offspring1=self._mutate(parents[0])
                    offspring2=self._mutate(parents[1])
                else:
                    # crossover
                    #print("CROSSOVER")
                    offspring1 = self._crossover(parents[0], parents[1])
                    offspring2 = self._crossover(parents[1], parents[0])
                
                next_gen.extend([offspring1, offspring2])

            
            next_gen = [(ind, cost) for ind, cost in sorted(next_gen, key=lambda x: x[1])]  
            population = next_gen[:self.pop_size]  # mantieni solo i migliori
        
            if population[0][1] < best_cost:
                best_chromo, best_cost = population[0]
                stagnation_counter = 0
            else:
                stagnation_counter += 1
            # se il costo rimane lo stess per 5 generazioni, interrompiamo
            if stagnation_counter >= 5:
               break

        return best_chromo

    def solution(self):
        best_chromo = self._run_ga_logic()
        return best_chromo, self._path_cost(best_chromo)

In [237]:
from Problem import Problem

p = Problem(100, density=0.2, alpha=1, beta=0.2)
print(f"Baseline Cost: {p.baseline()}")
    
solver = GA_Solver(p,  pop_size=10, generations=20, offprint=5)
best_path, best_cost = solver.solution()
print(f"\n\nGA Cost: {best_cost}")
print (f"Relative cost: {best_cost/p.baseline()*100:.2f}")

Baseline Cost: 653.9388659150875


GA Cost: 467.85390623001365
Relative cost: 71.54


In [238]:
from Problem import Problem

logging.getLogger().setLevel(logging.WARNING)

problems= [
    Problem(100, density=0.2, alpha=1, beta=0.2),
    Problem(100, density=0.2, alpha=0.5, beta=1),
    Problem(100, density=0.2, alpha=1, beta=2),
    Problem(100, density=1, alpha=1, beta=1),
    Problem(100, density=1, alpha=2, beta=1),
    Problem(100, density=1, alpha=30, beta=2),
    Problem(100, density=1, alpha=30, beta=0.1),
    Problem(100, density=1, alpha=0.1, beta=2),

]

for p in problems:
    print(f"\n\n--- PROBLEM: {p.graph.number_of_nodes()} cities, alpha {p.alpha}, beta {p.beta} ---")
    print(f"Baseline Cost: {p.baseline()}")
    
    solver = GA_Solver(p, pop_size=20, generations=50, offprint=10)
    best_path, best_cost = solver.solution()
    print(f"\n\nGA Cost: {best_cost}")
    print (f"Relative cost: {best_cost/p.baseline()*100:.2f}")



--- PROBLEM: 100 cities, alpha 1, beta 0.2 ---
Baseline Cost: 653.9388659150875


GA Cost: 469.9860795001688
Relative cost: 71.87


--- PROBLEM: 100 cities, alpha 0.5, beta 1 ---
Baseline Cost: 12686.953618676489


GA Cost: 14836.610859100992
Relative cost: 116.94


--- PROBLEM: 100 cities, alpha 1, beta 2 ---
Baseline Cost: 5334401.927002504


GA Cost: 250984.55635854986
Relative cost: 4.71


--- PROBLEM: 100 cities, alpha 1, beta 1 ---
Baseline Cost: 18266.185795826725


GA Cost: 19071.60815004375
Relative cost: 104.41


--- PROBLEM: 100 cities, alpha 2, beta 1 ---
Baseline Cost: 36457.91846237206


GA Cost: 37373.61823507845
Relative cost: 102.51


--- PROBLEM: 100 cities, alpha 30, beta 2 ---
Baseline Cost: 4864413346.733013


GA Cost: 173339014.99510017
Relative cost: 3.56


--- PROBLEM: 100 cities, alpha 30, beta 0.1 ---
Baseline Cost: 299.13938855299125


GA Cost: 287.0919451973951
Relative cost: 95.97


--- PROBLEM: 100 cities, alpha 0.1, beta 2 ---
Baseline Cost: 54123.48948

In [127]:
np.array(best_path)

array([[  0.        ,   0.        ],
       [ 16.        ,   0.        ],
       [ 17.        , 459.45685976],
       [ 16.        ,   0.        ],
       [  0.        ,   0.        ],
       [ 16.        ,   0.        ],
       [  5.        ,   0.        ],
       [  2.        , 832.84551786],
       [  3.        , 700.5648369 ],
       [  4.        , 313.05427474],
       [  5.        , 832.42754159],
       [  6.        , 804.95959314],
       [  7.        , 388.09090065],
       [  8.        , 289.03977583],
       [  9.        , 682.81300847],
       [ 10.        , 140.61273113],
       [  0.        ,   0.        ],
       [  1.        ,   0.        ],
       [  9.        ,   0.        ],
       [ 11.        , 200.70829427],
       [ 12.        ,   8.35490748],
       [ 13.        , 787.13745312],
       [ 14.        , 665.18600574],
       [ 15.        , 705.46021325],
       [ 16.        , 780.94830199],
       [  0.        ,   0.        ],
       [  1.        , 437.71476695],
 

In [215]:
def check_path(problem, path):
    total_cost = 0
    current_weight = 0
    is_correct=True
    total_gold= sum(problem.graph.nodes[ n ]['gold'] for n in problem.graph.nodes if n !=0 )

    for i in range(len(path)-1):
        u, v = path[i][0], path[i+1][0]
        d= problem.graph[u][v]['dist']
        total_cost += d + (problem.alpha * d * current_weight) ** problem.beta
        # Aggiorna il peso se si raccoglie oro
        if v != 0:  # Non raccogliere oro al deposito
            current_weight += path[i+1][1]
            total_gold -= path[i+1][1]
                
        else:
            current_weight = 0  # Svuota il peso al deposito
    if total_gold !=0:
        is_correct=False
    return total_gold, total_cost

calculated_cost = check_path(p, best_path)
print(f"Calcolated Cost from path: {calculated_cost}")

Calcolated Cost from path: (23751.399190151267, np.float64(18084.400859385958))
